In [1]:
import sys
sys.path.append('../..')

In [2]:
import pandas as pd

In [11]:
def appendPrefix(attribute,prefix):
    return list(map(lambda s:prefix+s,str(attribute).split()))

In [12]:
def preProcessNeighbors(nn,opposite_label_data,attribute):
    nn_values = pd.merge(nn,opposite_label_data,left_on=attribute,right_on='id')
    left_values, right_values  = nn_values['ltable_'+attribute],nn_values['rtable_'+attribute]
    left_values_prefixed = list(map(lambda att:tuple(appendPrefix(att,'L_')),left_values))
    right_values_prefixed = list(map(lambda att:tuple(appendPrefix(att,'R_')),right_values))
    return left_values_prefixed,right_values_prefixed

In [31]:
pos_nn = pd.read_csv('experiment_results/testpos_nn_hybrid_withfilter.csv')
train_neg = pd.read_csv('../../Structured/DBLP-ACM/train_negatives.csv')
lval,rval = preProcessNeighbors(pos_nn,train_neg,'title')
transactions = list(map(lambda l,r:l+r,lval,rval))

In [33]:
from efficient_apriori import apriori
itemsets, rules = apriori(transactions, min_support=0.05,  min_confidence=0.8)

rules_rhs = filter(lambda rule: len(rule.lhs) == 1 and len(rule.rhs)==1, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
  print(rule) # Prints the rule and its confidence, support, lift, ...

{R_for} -> {L_for} (conf: 0.893, supp: 0.307, lift: 2.375, conv: 5.849)
{L_for} -> {R_for} (conf: 0.817, supp: 0.307, lift: 2.375, conv: 3.586)
{L_systems} -> {L_database} (conf: 0.818, supp: 0.083, lift: 5.324, conv: 4.655)
{R_systems} -> {L_systems} (conf: 0.806, supp: 0.067, lift: 7.982, conv: 4.624)
{L_efficient} -> {R_efficient} (conf: 0.857, supp: 0.055, lift: 8.898, conv: 6.326)
{R_optimization} -> {L_optimization} (conf: 0.812, supp: 0.060, lift: 11.070, conv: 4.942)
{L_optimization} -> {R_optimization} (conf: 0.812, supp: 0.060, lift: 11.070, conv: 4.942)
{R_management} -> {L_management} (conf: 0.880, supp: 0.050, lift: 14.210, conv: 7.817)
{L_management} -> {R_management} (conf: 0.815, supp: 0.050, lift: 14.210, conv: 5.090)
